[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap04/cap04-main.ipynb)

# Capítulo 4 - Funções de Valor e Algoritmos de Monte Carlo


In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg freeglut3-dev xvfb

    !pip install gym==0.23.1

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )


In [ ]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

    from util.notebook import display_videos_from_path

In [ ]:
import gym
import numpy as np

from util.plot import plot_result
from util.experiments import test_greedy_Q_policy

## 1 - Introdução

Vamos relembrar alguns conceitos da aula passada que serão muito úteis nesta aula.


### 1.1 Retornos Parciais Descontados

Os retornos parciais são calculados a partir de um passo $t$ qualquer da trajetória:

$$
\begin{align*}
   && &\quad G_0     &= &\;\;R_1 &+ \gamma &R_2 &+ \gamma^2 &R_3 &+ \gamma^3 &R_4 &+ \gamma^4 &R_5 &+ \;\cdots \;&+ \gamma^{T-1} &R_T & \;\;\;(= G) \\
   && &\quad G_1     &= &        &         &R_2 &+ \gamma   &R_3 &+ \gamma^2 &R_4 &+ \gamma^3 &R_5 &+ \;\cdots \;&+ \gamma^{T-2} &R_T &       \\
   && &\quad G_2     &= &        &         &    &           &R_3 &+ \gamma   &R_4 &+ \gamma^2 &R_5 &+ \;\cdots \;&+ \gamma^{T-3} &R_T &       \\
   && &\quad         &\cdots &   & & & & & & & & & & &    & \\
   && &\quad G_T     &= &        & & & & & & & & & & &0   & \\
\end{align*}
$$

Lembrando que, para $t < T$ qualquer, observamos esta relação:

$$
   G_{t} = R_{t+1} + \gamma G_{t+1}
$$

### 1.2 - Função de valor do estado $V(s)$

Esta função dá o retorno parcial esperado a partir de cada estado $s$, para uma política específica.

De forma matemática, ela é definida assim:

$$V(s) = E[G_t | S_t=s]$$

### 1.3 - Função de valor da ação $Q(s,a)$

Esta é a função mais importante para esta aula.

O $Q(s,a)$ dá o valor médio do retorno parcial a partir do par estado-ação $(s,a)$:

$$Q(s,a) = E[G_t | S_t=s, A_t=a]$$

De maneira informa, ele responde a esta pergunta:

*Quando estava no estado* **s** *e fez a ação* **a** *, qual o retorno esperado (se continuar seguindo a mesma política)?*


### 1.4 Algoritmos de Monte Carlo

***O que são algoritmos de Monte Carlo?***

 - Utilizam amostragem aleatória para resolver problemas ou estimar quantidades que são difíceis de calcular diretamente.
 - Envolvem a geração de um grande número de amostras ou simulações aleatórias para tirar conclusões ou fazer previsões.
 - Usados em várias áreas (não só na Aprendizagem por Reforço)

Vimos, antes, dois algoritmos de Monte Carlo para o problema de **predição** da aprendizagem por reforço.
- Para aprender os valores de $V(s)$ e de $Q(s,a)$.

Aqui, veremos algoritmos de Monte Carlo para o problema de **controle** da aprendizagem por reforço.
- Para aprender a política ótima!

Os algoritmos de controle que veremos se baseiam na **função de valor da ação `Q(s,a)`**.

## 2 - Algoritmo (de Controle) de Monte Carlo - Versão 1

Este algoritmo roda vários episódios, fazendo estes passos a cada episódio:

1. Gera a **trajetória** completa (sequência de estados/observações, ações e recompensas) do episódio:

    $S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{n-1} \rightarrow A_{n-1} \rightarrow R_T \rightarrow S_T$

1. Para escolher a ação $a$, a ser realizada em um estado $s$, ele usa a tabela $Q(s,a)$ com alguma estratégia de exploração -- vamos usar $\epsilon$-greedy. Assim, ele escolhe dessa forma:
   - com probabilidade $\epsilon$, ele escolhe uma ação $a$ qualquer
   - com probabilidade $1-\epsilon$, ele escolhe a melhor ação, ou seja, $\max_a{Q(s,a)}$

1. Ao fim do episódio, para cada par intermediário ($S_t$, $A_t$) da trajetória, ele:
   - calcula o retorno parcial $G_t$.
   - usa $G_t$ para atualizar $Q(S_t, A_t)$


In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def choose_action_1(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])   # alt. para aleatorizar empates: np.random.choice(np.where(b == bmax)[0])

In [ ]:
def generate_and_show_video(env, Q):
    num_actions = env.action_space.n
    video_path = '/content/videos'
    # roda dois episódios, gravando o resultado
    rec_env = gym.wrappers.RecordVideo(env, video_path)
    for _ in range(2):
      state = rec_env.reset()
      done = False
      while not done:
          action = choose_action_1(Q, state, num_actions, 0.0)
          next_state, reward, done, _ = rec_env.step(action)
    display_videos_from_path(video_path)
    rec_env.close()

In [ ]:
# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarlo1(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    num_actions = env.action_space.n

    # dicionário com todos os retornos descontados, para cada par (estado,ação)
    returns_history = dict()

    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []

        state = env.reset()

        # [1] Executa um episódio completo, salvando a trajetória
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0) and not IN_COLAB:
                env.render()

            # [2] Escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action_1(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # [3] Calcula os retornos parciais e atualiza Q
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt

            if returns_history.get((s,a)) is None:
                returns_history[s,a] = [ Gt ]
            else:
                returns_history[s,a].append(Gt)

            # média entre todas as ocorrências de (s,a) encontradas nos episódios
            Q[s,a] = np.mean(returns_history[s,a]) # LENTO!
            # IDEIA: usar cálculo alternativo da média: M = M + (1/n)*(x - M)

    # para gerar um vídeo no Colab
    if render and IN_COLAB:
        clear_output()
        generate_and_show_video(env, Q)

    return sum_rewards_per_ep, Q

In [ ]:
%timeit
# estes dois ambientes possuem estados e ações discretas
# ver mais em: https://www.gymlibrary.dev/
#ENV_NAME = "Taxi-v3"
ENV_NAME = "FrozenLake-v1"

# usar 10 para Taxi e 1.0 para FrozenLake
r_max_plot = 10.0

EPISODES = 1000
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
rewards, Qtable = run_montecarlo1(env, EPISODES, GAMMA, EPSILON, render=True)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))


In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
# Se quiser salvar, passe o nome do arquivo no 3o parâmetro
filename = f"results/montecarlo1-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot, None)

## 3 - Algoritmo (de Controle) de Monte Carlo - Versão 2

Modifique o código acima com essas características:
1. Mantenha o uso de `choose_action_1()` para escolher a ação.
1. Remova o **histórico** de retornos parciais.
1. Use uma **taxa de aprendizagem**, representada pelo parâmetro `lr`.

Faça as melhorias abaixo:

In [ ]:
# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarlo2(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []

        state = env.reset()

        # [1] Executa um episódio completo, salvando a trajetória
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0) and not IN_COLAB:
                env.render()

            # [2] Escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action_1(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # [3] Calcula os retornos parciais e atualiza Q
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            # FAÇA AQUI A MODIFICAÇÂO PEDIDA !!!

    return sum_rewards_per_ep, Q

Quando o código estiver pronto, você poderá testar rodando o código abaixo:

In [ ]:
ENV_NAME = "Taxi-v3"
#ENV_NAME = "FrozenLake-v1"

r_max_plot = 10.0

EPISODES = 20000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
rewards, Qtable = run_montecarlo2(env, EPISODES, LR, GAMMA, EPSILON, render=True)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
# Se quiser salvar, passe o nome do arquivo no 3o parâmetro
#filename = f"results/montecarlo2-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot, None)

## 4 - Experimentos Finais


In [ ]:
from util.experiments import repeated_exec
from util.plot import plot_multiple_results

Vamos comparar os algoritmos aqui propostos.

In [ ]:
NUM_EPISODES = 5000

#enviroment = gym.make("Taxi-v3")
enviroment = gym.make("FrozenLake-v1")

In [ ]:
results = []

# muito lento, se usar o histórico completo!
#results.append( repeated_exec(1, "Monte-Carlo1", run_montecarlo1, enviroment, NUM_EPISODES) )

for learning_rate in [0.01, 0.1, 0.5]:
    results.append( repeated_exec(10, f"Monte-Carlo2 (LR={learning_rate})", run_montecarlo2, enviroment, NUM_EPISODES, learning_rate) )

clear_output()

In [ ]:
plot_multiple_results(results, cumulative=False, x_log_scale=False, window=50)